Предварительно нужно **запулить docker-image** и **локально запустить контейнер**. Используйте последовательно в терминале следующие команды:
1. docker image pull bigfloppa228/1984_db:latest
2. docker run -p 6333:6333 bigfloppa228/1984_db:latest

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-large',
        model_kwargs={"device": "cuda"}
    )

In [2]:
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient

qdrant_client = QdrantClient("localhost", port=6333, timeout=100)
doc_store = Qdrant(
    qdrant_client, 
    embeddings=embeddings,
    collection_name='qdrant_novels_1984',
    content_payload_key='page_content',
    metadata_payload_key='metadata',
    )

In [3]:
prefixes = ('query: ', 'passage: ')
retriever = doc_store.as_retriever(search_type='mmr', search_kwargs={"k": 5})

In [4]:
queries = [
    'Что такое ангсоц?',
    'Кто является главным злодеем произведения?',
    'Перечисли основные министерства Англии.',
    'Расскажи о Джулии.'
]

In [5]:
for query in queries:
    print(f'Query: {query}')
    print('=' * 90)
    print(f'Best candidates:')
    
    best_segments = retriever.get_relevant_documents(prefixes[0] + query)
    
    for i, elem in enumerate(best_segments):
        print(f'{i+1}. {elem.metadata["context"]}')
        
    print()

Query: Что такое ангсоц?
Best candidates:
1. Теперь их падало на Лондон по двадцать-тридцать штук в неделю. Внизу на улице ветер трепал рваный плакат, на нем мелькало слово АНГСОЦ. Ангсоц. Священные устои ангсоца. Новояз, двоемыслие, зыбкость прошлого. У него возникло такое чувство, как будто он бредет по лесу на океанском дне, заблудился в мире чудищ и сам он — чудище. Он был один. Прошлое умерло, будущее нельзя вообразить. Есть ли какая-нибудь уверенность, что хоть один человек из живых — на его стороне?
2. Считалось, что, если класс капиталистов лишить собственности, наступит социализм; и капиталистов, несомненно, лишили собственности. У них отняли все — заводы, шахты, землю, дома, транспорт; а раз все это перестало быть частной собственностью, значит, стало общественной собственностью. Ангсоц, выросший из старого социалистического движения и унаследовавший его фразеологию, в самом деле выполнил главный пункт социалистической программы — с результатом, который он предвидел и к котор

In [6]:
def search(query, sentences_number=5, score_threshold=0.8):
    retriever = doc_store.as_retriever(search_type='mmr', search_kwargs={"k": sentences_number, "score_threshold": float(score_threshold)})
    return retriever.get_relevant_documents('query: ' + query)

In [7]:
search('Кто является главным злодеем произведения?', 10, 0.4)

[Document(page_content='passage: Первый изменник, главный осквернитель партийной чистоты.', metadata={'context': 'Голдстейн, отступник и ренегат, когда-то, давным-давно (так давно. что никто уже и не помнил, когда), был одним из руководителей партии, почти равным самому Старшему Брату, а потом встал на путь контрреволюции, был приговорен к смертной казни и таинственным образом сбежал, исчез. Программа двухминутки каждый день менялась, но главным действующим лицом в ней всегда был Голдстейн. Первый изменник, главный осквернитель партийной чистоты. Из его теорий произрастали все дальнейшие преступления против партии, все вредительства, предательства, ереси, уклоны. Неведомо где он все еще жил и ковал крамолу: возможно, за морем, под защитой своих иностранных хозяев, а возможно — ходили и такие слухи, — здесь, в Океании, в подполье.', '_id': '62844aef-a5a1-4bb7-8e99-6428b8f2e9d0', '_collection_name': 'qdrant_novels_1984'}),
 Document(page_content='passage: Тогда тайное отвращение к Старше

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

checkpoint = 'openchat/openchat-3.5-0106'

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

D:\Dev\ml_models\venv\Lib\site-packages\transformers\generation\configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [13]:
from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

pipeline_kwargs = {"max_new_tokens": 512, "do_sample": True, "temperature": 0.2, "top_k": 30, "top_p": 0.9, "repetition_penalty": 1.05, "pad_token_id": 32000}
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, **pipeline_kwargs)
hugging_face_pipeline = HuggingFacePipeline(pipeline=pipe)

def postprocess(generation):
    return generation[generation.find('GPT4 Correct Assistant:') + len('GPT4 Correct Assistant:'):].strip()

def search_and_generate(query, sentences_number=5, score_threshold=0.8, use_rag=True):
    relevant_docs = search(query, sentences_number, score_threshold)
    
    context = []
    for i, doc in enumerate(relevant_docs):
        context.append(f'{i + 1}. {doc.metadata["context"]}')
    prompt_context = '\n'.join(context)

    system_part = f"""You are a literary theorist who has the high expertise in the writings of George Orwell.
Your task is to answer the question about his dystopian novel '1984'.
Write everything you know about the question.
"""
    rag_part = f"""Use the following relevant information during the answer.

Relevant information:
{prompt_context}
""" 
    question_part = f"""Question: {query}"""

    # no RAG
    if not use_rag:
        prompt = '\n'.join([system_part, question_part])
    # use RAG
    else:
        prompt = '\n'.join([system_part, rag_part, question_part])

    chat = [{"role": "user", "content": prompt}]

    chain = hugging_face_pipeline | postprocess
    return relevant_docs, chain.invoke(tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True))

In [14]:
relevant_docs, answer = search_and_generate('Кто является главным злодеем произведения?', 5, 0.4, False)
print(answer)

В произведении "1984" Джорджа Орвелла главным злодеем является враг народа Океании, Большой Брат. Он является лидером и диктатором страны Океанию, в которой действие происходит, и символизирует тоталитарную власть и контроль над людьми. Большой Брат не только манипулирует общественными мнениями и управляет жизнью каждого гражданина через систему смотрителей, но и заставляет людей верить в свою присутственность, используя технологии мониторинга и манипуляции фактами. В этом произведении Орвелл показывает, как тоталитарная власть может исказить реальность и ограничить свободу человека, делая из него подданного, готового следовать безразличным к причинам приказам.


In [16]:
relevant_docs, answer = search_and_generate('Кто является главным злодеем произведения?', 5, 0.4, True)
print(answer)

Главным злодеем произведения "1984" является Старший Брат. Он является лидером Океании и символизирует тиранию и диктатуру. Его правление характеризуется ненавистью к мыслям, подавлением свободы и идеологической чистоты. Старший Брат является центральной фигурой, которая определяет судьбу героев и создает атмосферу страха и непостоянства в обществе.


In [17]:
relevant_docs

[Document(page_content='passage: Первый изменник, главный осквернитель партийной чистоты.', metadata={'context': 'Голдстейн, отступник и ренегат, когда-то, давным-давно (так давно. что никто уже и не помнил, когда), был одним из руководителей партии, почти равным самому Старшему Брату, а потом встал на путь контрреволюции, был приговорен к смертной казни и таинственным образом сбежал, исчез. Программа двухминутки каждый день менялась, но главным действующим лицом в ней всегда был Голдстейн. Первый изменник, главный осквернитель партийной чистоты. Из его теорий произрастали все дальнейшие преступления против партии, все вредительства, предательства, ереси, уклоны. Неведомо где он все еще жил и ковал крамолу: возможно, за морем, под защитой своих иностранных хозяев, а возможно — ходили и такие слухи, — здесь, в Океании, в подполье.', '_id': '62844aef-a5a1-4bb7-8e99-6428b8f2e9d0', '_collection_name': 'qdrant_novels_1984'}),
 Document(page_content='passage: Тогда тайное отвращение к Старше